Pour les années à étudier
Créer toeval, càd csv avec intéret

# Useful comments

295 793 files </br>
Class labels in json : 'main_cpc_label', 'cpc_labels', 'main_ipcr_label', 'ipcr_labels', 'uspc_class', 'uspc_subclass' </br>
IDs : 'patent_number' </br>
text info : 'abstract', 'claims', 'background', 'summary', 'full_description', 'title' </br>
Innovative label : 'decision'  </br>

 
Patent Acceptance Prediction. Given a section of an application (in particular, the abstract, claims,
or description), we predict whether the application will be accepted by the USPTO. From the
perspective of the NLP community, this is a standard classification task. Yet, the potential applications
and benefits of this decision task, as well as its difficulty, distinguish it from prevalent binary
classification benchmarks (e.g., SST, Yelp). In our experiments, we focus on applications without
parent filings to make our setup simple and clear, thereby excluding all the CONT-applications. Also,
we do not include any pending applications

# First collection

Uploading the years for evaluation - 2016/2015/2014 with the classes to evaluate H01L/G06F/A61B

In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
import collections
import re
from tqdm import tqdm

In [51]:
mypath = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/data/2004/"  #'data/2016/2016/'
_2014_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

len(_2014_files)

# _2014_files est une liste de string

207936

In [21]:
for file in _2014_files[:5]:
    print(file)

print(_2014_files[:5])

10018320.json
10018639.json
10048553.json
10049016.json
10066553.json
['10018320.json', '10018639.json', '10048553.json', '10049016.json', '10066553.json']


In [28]:
int()
_2014_files_light = _2014_files[:5000]
_2014_files_light
mypath

'C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/data/2004/'

In [29]:
# Check on all decision and repartition -- I think we should keep only the one accepted based on other USPTO patent (continuity)
decision_H01L = collections.defaultdict(int)
decision_G06F = collections.defaultdict(int)
decision_A61B = collections.defaultdict(int)
patent_H01L_list = []
patent_G06F_list = []
patent_A61B_list = []
for i in tqdm(range(len(_2014_files_light))):
    patent_path = mypath + _2014_files_light[i]
    with open(patent_path) as f:
        d = json.load(f)
        f.close()
    class_mainIPC = d['main_ipcr_label']
    if re.match(r'^H01L', class_mainIPC):
        patent_H01L_list.append(_2014_files_light[i])
        decision_H01L[d['decision']] += 1
    if re.match(r'^G06F', class_mainIPC):
        patent_G06F_list.append(_2014_files_light[i])
        decision_G06F[d['decision']] += 1
    if re.match(r'^A61B', class_mainIPC):
        patent_A61B_list.append(_2014_files_light[i])
        decision_A61B[d['decision']] += 1

100%|██████████| 5000/5000 [01:09<00:00, 71.65it/s] 


In [32]:
print(len(patent_H01L_list), len(patent_G06F_list), len(patent_A61B_list))
print(decision_H01L, decision_G06F, decision_A61B)

7 33 5
defaultdict(<class 'int'>, {'ACCEPTED': 5, 'REJECTED': 2}) defaultdict(<class 'int'>, {'REJECTED': 18, 'ACCEPTED': 15}) defaultdict(<class 'int'>, {'ACCEPTED': 2, 'REJECTED': 3})


In [31]:
patent_H01L_list

['10451564.json',
 '10482328.json',
 '10482350.json',
 '10483212.json',
 '10486666.json',
 '10490598.json',
 '10492726.json']

In [10]:
# Saving list of patent_filename into list
#with open(r'data/G01_filename_2016.txt', 'w') as fp:
#    for item in patent_G01N_list:
#        # write each item on a new line
#        fp.write("%s\n" % item)
#    print('Done')

Done


In [42]:
#Importing file for patent_list 2016
#patent_G01_list = []
#with open(r'data/G01_filename_2016.txt', 'r') as fp:
#    for line in fp:
#        x = line[:-1]
#        patent_G01_list.append(x)
#        
#print(len(patent_G01_list))

6834


In [33]:
final_patents = []
decision = []
for i in tqdm(range(len(patent_A61B_list))):
    patent_path = mypath + patent_A61B_list[i]
    with open(patent_path) as f:
        d = json.load(f)
        f.close()
    if d['decision'] == 'ACCEPTED' or d['decision'] == 'REJECTED': # exclure pending
        final_patents.append(patent_A61B_list[i])
        decision.append(d['decision'])

100%|██████████| 5/5 [00:00<00:00, 2293.22it/s]


In [34]:
set(decision)

{'ACCEPTED', 'REJECTED'}

In [35]:
len(final_patents)

5

In [1]:
pprint(final_patents)

NameError: name 'final_patents' is not defined

In [45]:
# Saving list of patent_filename into list
#with open(r'data/G01_filename_2016_notpending.txt', 'w') as fp:
 #   for item in final_patents:
 #       # write each item on a new line
 ##       fp.write("%s\n" % item)
 #   print('Done')

Done


# Récuéparation des éléments

In [46]:
#Importing file for patent_list 2017
#patent_G01_list = []
#with open(r'data/G01_filename_2016_notpending.txt', 'r') as fp:
#    for line in fp:
#        x = line[:-1]
 #       patent_G01_list.append(x)
        
#print(len(patent_G01_list))

1979


In [37]:
non_main_ipc = []
patent_number, labels, titles, backgrounds, claims = [], [], [], [], []
for i in tqdm(range(len(final_patents))):
    patent_path = mypath + final_patents[i]
    with open(patent_path) as f:
        d = json.load(f)
        f.close()
    
    #Creating the lists for the other information
    patent_number.append(d['application_number'])
    titles.append(d['title'])
    backgrounds.append(d['background'])
    claims.append(d['claims'])
    
    #Collecting non main ipc class
    non_main =  d['ipcr_labels']
    for ipcr in non_main:
        non_main_ipc.append(ipcr) # only 4 first characters to be sure of being at same level
    #Getting labels based on decision
    label = 0
    if d['decision'] == 'ACCEPTED':
        label = 1
    labels.append(label)

100%|██████████| 5/5 [00:00<00:00, 2341.88it/s]


In [40]:
expectations_classes = list(set(non_main_ipc))
len(expectations_classes)
expectations_classes

['A61B814',
 'G06Q1000',
 'A61B300',
 'A61B500',
 'A61B1818',
 'A61M2700',
 'A61F1100']

In [41]:
#Removing classes of the main classe
good_expectations_classes = []
for ipcr in expectations_classes:
    if ipcr[0:4] != "A61B":
        good_expectations_classes.append(ipcr)
        
print(len(good_expectations_classes))
good_expectations_classes

3


['G06Q1000', 'A61M2700', 'A61F1100']

In [32]:
#saving expectations class in txt file
with open(r'data/expectation_space/2014_A61B_expectation_IPC_class.txt', 'w') as fp:
    for item in good_expectations_classes:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [42]:
df = pd.DataFrame({'application_number': patent_number, 'title': titles, 'background': backgrounds, 'claims':claims, 'label': labels})

In [43]:
len(df)

5

In [50]:
pd.options.display.max_colwidth = 200
print(df.claims[:1])

0    1. An apparatus for intravascular ultrasonic imaging, the apparatus comprising: a catheter having an ultrasonic transducer array fabricated at least in part from non-polymeric electrostrictive mat...
Name: claims, dtype: object


In [35]:
df.to_csv('data/to_eval/2014_A61B_patents_toeval.csv', index=False)